# Segmentação e Taxa de Retenção de Clientes

O objetivo deste notebook é segmentar os clientes presentes na base de dados e calcular a taxa de rentenção de clientes com base na frequência de pedido.

## Importação de Bibliotecas

In [31]:
import pandas as pd


## Importação de Dados

In [32]:
df = pd.read_csv('../data/clean_general_df.csv')

## Preparação dos Dados

In [33]:
# Converter colunas de data em datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

In [34]:
# Pedidos entregues com data válida
df = df[df['order_status'] != 'canceled']
df = df.dropna(subset=['order_purchase_timestamp', 'payment_value'])

## Segmentação

In [35]:
rfm = df.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (df['order_purchase_timestamp'].max() - x.max()).days,
    'order_id': 'nunique',
    'payment_value': 'sum'
}).reset_index()

rfm.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']


In [36]:
# Atribuir scores RFM (1 = pior, 4 = melhor)
rfm['recency_score'] = pd.qcut(rfm['Recency'], q= 5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method="first"), q= 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm['monetary_score'] = pd.qcut(rfm['Monetary'], q= 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm['freq_and_mon_score'] = ((rfm['frequency_score'] + rfm['monetary_score']) / 2).round().astype(int)

Os clusters para a segmentação do clientes serão construídos com base no mapeamento do Modelo RFM de segmentação de clientes, conforme o mapa abaixo:

![segment-map](../img/segment-map.jpg)

In [37]:
def segment_customers(rfm):
    """
    Segmenta clientes com base em uma matriz R x FM usando mapeamento direto.
    """
    # Tabela de segmentação baseada no gráfico
    segments_map = {
        (5, 5): 'Champions',
        (5, 4): 'Loyal Customers',
        (5, 3): 'Loyal Customers',
        (5, 2): 'New Customers',
        (5, 1): 'New Customers',
        (4, 5): 'Potential Loyalist',
        (4, 4): 'Potential Loyalist',
        (4, 3): 'Potential Loyalist',
        (4, 2): 'Promising',
        (4, 1): 'Promising',
        (3, 5): 'Loyal Customers',
        (3, 4): 'Loyal Customers',
        (3, 3): 'Need Attention',
        (3, 2): 'About To Sleep',
        (3, 1): 'About To Sleep',
        (2, 5): "Can't Loose Them",
        (2, 4): "Can't Loose Them",
        (2, 3): "At Risk",
        (2, 2): 'Hibernating',
        (2, 1): 'Hibernating',
        (1, 5): "Can't Loose Them",
        (1, 4): "Can't Loose Them",
        (1, 3): "At Risk",
        (1, 2): 'Hibernating',
        (1, 1): 'Lost'
    }

    def classify(row):
        return segments_map.get((row['recency_score'], row['freq_and_mon_score']))

    rfm['segments'] = rfm.apply(classify, axis=1)
    return rfm

In [38]:
rfm = segment_customers(rfm)

rfm['segments'].value_counts(ascending= False)

segments
Loyal Customers       17897
Can't Loose Them      14985
Hibernating           14854
Potential Loyalist    11536
About To Sleep         7821
New Customers          7783
Promising              7675
At Risk                7427
Need Attention         3784
Champions               982
Lost                    815
Name: count, dtype: int64

Ao alocar os clientes nos clusters do modelo RFM, podemos criar estratégias sólidas de marketing para cada cluster:
- **Campeões**: Oferecer recompensas exclusivas e antecipar lançamentos.
- **Clientes Leais**: Implementar programa de fidelidade com benefícios progressivos.
- **Não posso perder**: Enviar campanhas personalizadas de reengajamento com ofertas especiais.
- **Hibernando**: Lançar promoções de “última chance” para estimular retorno.
- **Clientes Fiéis em Potencial**: Incentivar a fidelização com cashback ou frete grátis.
- **Quase dormindo**: Disparar e-mails de reativação com produtos recomendados.
- **Clientes Novos**: Oferecer cupom de desconto para a segunda compra.
- **Promessas**: Enviar sugestões de produtos complementares.
- **Em Risco**: Criar campanhas urgentes com ofertas exclusivas e limitadas.
- **Precisa de Atenção**: Aplicar pesquisa rápida para entender e recuperar o interesse.
- **Perdidos**: Enviar última campanha de recuperação com oferta muito atrativa.

## Taxa de Retenção de Clientes

In [39]:
customers_group = df.groupby('customer_unique_id')['order_id'].nunique().reset_index()
returned_customers = customers_group[customers_group['order_id'] > 1]

returned_customers_rate = round((returned_customers.shape[0] / customers_group.shape[0]) * 100, 2)
print(f'A taxa de retenção de clientes é de {returned_customers_rate}%')

A taxa de retenção de clientes é de 3.06%


A taxa de retenção de clientes é extremamente pequena, evidenciando que o e-commerce tem problemas de manter os seus clientes ativos e frequentes. Esse fato é constatado ao observar a quantidade expressiva de clientes em segmentos de baixa frequência e recência.